## Setup

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### Instalação

In [ ]:
#!pip install transformers==4.20.0
!pip install --no-cache-dir transformers sentencepiece
!pip install keras_nlp==0.3.0
!pip install datasets
!pip install huggingface-hub
!pip install nltk

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 167.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 174.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 331.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 166.2 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.4/142.4 KB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 54.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 588.3/588.3 MB 2.8 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of tensorflow-text to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.9/5.9 MB 97.4 MB/s eta 0:00:00
     ━━━━

### Imports

In [ ]:
import os
import logging

import nltk
import numpy as np
import tensorflow as tf
from tensorflow import keras

tf.get_logger().setLevel(logging.ERROR)

os.environ["TOKENIZERS_PARALLELISM"] = "false"

### Configurações para o fine-tunning

In [ ]:
TRAIN_TEST_SPLIT = 0.1

MAX_INPUT_LENGTH = 1024
MIN_TARGET_LENGTH = 5
MAX_TARGET_LENGTH = 512
BATCH_SIZE = 8
LEARNING_RATE = 1e-4
MAX_EPOCHS = 5


MODEL_CHECKPOINT = "unicamp-dl/ptt5-small-portuguese-vocab" #T5 small pré-treinado em portugûes, utilizando o BrWac corpus

## Download do dataset

Será utilizado o dataset [XL-Sum](https://github.com/csebuetnlp/xl-sum) . Ele possui artigos brasileiros da BBC, juntamente com seus resumos, contendo 57402 artigos.

A função load_dataset do Hugging Face será utilizada para carregar o dataset.

In [ ]:
from datasets import load_dataset

In [ ]:
!tar xjvf /content/drive/MyDrive/portuguese_XLSum_v2.0.tar.bz2

./
./portuguese_test.jsonl
./portuguese_val.jsonl
./portuguese_train.jsonl


In [ ]:
dataset = load_dataset('json', data_files='/content/portuguese_train.jsonl',split='train')

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset json downloaded and prepared to /root/.cache/huggingface/datasets/json/default-f2e75642744a6add/0.0.0/0f7e3662623656454fcd2b650f34e886a7db4b9104504885bd462096cc7a9f51. Subsequent calls will reuse this data.


In [ ]:
dataset

Dataset({
    features: ['id', 'url', 'title', 'summary', 'text'],
    num_rows: 57402
})

Features presentes no dataset:

- **text**: o artigo em si
- **summary**: o resumo do artigo
- **id**: o ID do par artigo-resumo
- **url**: Link para o artigo
- **title**: Título do artigo

In [ ]:
dataset[0]

{'id': '121217_eua_enterro_massacre_escola_lgb',
 'url': 'https://www.bbc.com/portuguese/ultimas_noticias/2012/12/121217_eua_enterro_massacre_escola_lgb',
 'title': 'EUA enterram mortos de massacre em escola',
 'summary': 'A comunidade de Newtown deu início nesta segunda-feira ao enterro das 26 vítimas do massacre na escola Sandy Hook School, no Estado americano de Connecticut.',
 'text': 'Na última sexta-feira, um atirador abriu fogo dentro da instituição, matando 20 crianças e seis adultos. Mais cedo, a polícia de Connecticut disse que havia começado a analisar evidências - como o exame de balística - e que continuava ouvindo testemunhas da tragédia. O porta-voz da polícia, Paul Vance, disse que os adultos que sobreviveram ao tiroteio estavam se recuperando e que as escolas próximas reabririam na próxima terça-feira.'}

Dividindo os dados em 90% treino e 10% teste

In [ ]:
raw_datasets = dataset.train_test_split(
    train_size=1-TRAIN_TEST_SPLIT, test_size=TRAIN_TEST_SPLIT
)

## Pré-processamento dos dados


Como o T5 será o modelo em que o fine-tunning será aplicado, o tokenizador a ser utilizada também é o utilizado no T5. Através do tokenizador, o T5 irá tokenizar o texto com base em seu vocabulário, além de já converter os tokens para seus respectivos id's.


In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(MODEL_CHECKPOINT,use_fast=False)

Como o T5 é capaz de executar diversas tasks em um formato text-to-text, aqui é definido o prefixo a ser colocado antes do texto de input, indicando que o T5 deve resumir tal texto.

In [ ]:
prefix = "summarize: "

A função a seguir é responsável por adicionar o prefixos aos textos de entrada, tokenizar os textos de entrada, mas também tokenizar a variável alvo, que são os resumos.

In [ ]:
def preprocess_function(examples):
    inputs = [prefix + doc for doc in examples["text"]]
    model_inputs = tokenizer(inputs, max_length=MAX_INPUT_LENGTH, truncation=True)

    # Setup the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(
            examples["summary"], max_length=MAX_TARGET_LENGTH, truncation=True
        )

    model_inputs["labels"] = labels["input_ids"]

    return model_inputs


Em seguida, o dataset carregado é transformado utilizando a função definida acima.

In [ ]:
tokenized_datasets = raw_datasets.map(preprocess_function, batched=True)

  0%|          | 0/52 [00:00<?, ?ba/s]

/usr/local/lib/python3.8/dist-packages/transformers/tokenization_utils_base.py:3581: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


  0%|          | 0/6 [00:00<?, ?ba/s]

## Criando o modelo

Como será uma tarefa textual em que uma sequência é convertida em outra sequência, a classe a ser utilizada para carregar o modelo em que o fine-tunning será aplicado será a TFAutoModelForSeq2SeqLM.

In [ ]:
from transformers import TFAutoModelForSeq2SeqLM, DataCollatorForSeq2Seq

model = TFAutoModelForSeq2SeqLM.from_pretrained(MODEL_CHECKPOINT)

All model checkpoint layers were used when initializing TFT5ForConditionalGeneration.

All the layers of TFT5ForConditionalGeneration were initialized from the model checkpoint at unicamp-dl/ptt5-small-portuguese-vocab.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFT5ForConditionalGeneration for predictions without further training.


O data collator abaixo será utilizado para dividir os dados em batches, além de aplicar padding caso seja necessário.

In [ ]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model, return_tensors="tf")

Abaixo os dados são divididos em treino, teste e geração. Os dados de geração são utilizados para calcular a métrica ROUGE durante o treinamento.

Além disso, os dados também são divididos em batches.

In [ ]:
train_dataset = tokenized_datasets["train"].to_tf_dataset(
    batch_size=BATCH_SIZE,
    columns=["input_ids", "attention_mask", "labels"],
    shuffle=True,
    collate_fn=data_collator,
)
test_dataset = tokenized_datasets["test"].to_tf_dataset(
    batch_size=BATCH_SIZE,
    columns=["input_ids", "attention_mask", "labels"],
    shuffle=False,
    collate_fn=data_collator,
)
generation_dataset = (
    tokenized_datasets["test"]
    .shuffle()
    .select(list(range(200)))
    .to_tf_dataset(
        batch_size=BATCH_SIZE,
        columns=["input_ids", "attention_mask", "labels"],
        shuffle=False,
        collate_fn=data_collator,
    )
)

Compilando o modelo:

In [ ]:
optimizer = keras.optimizers.Adam(learning_rate=LEARNING_RATE)
model.compile(optimizer=optimizer)

No loss specified in compile() - the model's internal loss computation will be used as the loss. Don't panic - this is a common way to train TensorFlow models in Transformers! To disable this behaviour please pass a loss argument, or explicitly pass `loss=None` if you do not want your model to compute a loss.


## Treino e avaliação do modelo

O modelo será avaliado de acordo com as métricas Rouge-1, Rouge-2 e Rouge-L.

In [ ]:
!pip install rouge

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from rouge import Rouge

Função que irá calcular as métricas durante o treinamento:

In [ ]:
import keras_nlp
#import rouge_score

#rouge_l = keras_nlp.metrics.RougeL()
rouge = Rouge()


def metric_fn(eval_predictions):
    predictions, labels = eval_predictions
    decoded_predictions = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    for label in labels:
        label[label < 0] = tokenizer.pad_token_id
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    #result = rouge_l(decoded_labels, decoded_predictions)
    result = rouge.get_scores(decoded_predictions, decoded_labels, avg=True)
    
    result = {"Rouge-1": result["rouge-1"]["f"], "Rouge-2": result["rouge-2"]["f"], "RougeL": result["rouge-l"]["f"]}

    return result


Iniciando o treinamento:

In [ ]:
from transformers.keras_callbacks import KerasMetricCallback

metric_callback = KerasMetricCallback(
    metric_fn, eval_dataset=generation_dataset, predict_with_generate=True
)

callbacks = [metric_callback]


model.fit(
    train_dataset, validation_data=test_dataset, epochs=MAX_EPOCHS, callbacks=callbacks
)

Epoch 1/5
718/718 [==============================] - 814s 1s/step - loss: 3.6584 - val_loss: 3.0198 - Rouge-1: 0.1845 - Rouge-2: 0.0364 - RougeL: 0.1526
Epoch 2/5
718/718 [==============================] - 772s 1s/step - loss: 3.2061 - val_loss: 2.9489 - Rouge-1: 0.1918 - Rouge-2: 0.0452 - RougeL: 0.1603
Epoch 3/5
718/718 [==============================] - 774s 1s/step - loss: 3.0968 - val_loss: 2.9077 - Rouge-1: 0.1945 - Rouge-2: 0.0458 - RougeL: 0.1631
Epoch 4/5
718/718 [==============================] - 794s 1s/step - loss: 3.0215 - val_loss: 2.8885 - Rouge-1: 0.2054 - Rouge-2: 0.0558 - RougeL: 0.1737
Epoch 5/5
718/718 [==============================] - 771s 1s/step - loss: 2.9509 - val_loss: 2.8712 - Rouge-1: 0.1949 - Rouge-2: 0.0483 - RougeL: 0.1597


Salvando o modelo treinado:

In [ ]:
model.save_weights('/content/drive/MyDrive/ptt5_summary_model_weights.h5')

## Inferência

In [ ]:
model.load_weights('/content/drive/MyDrive/ptt5_summary_model_weights.h5')

In [ ]:
from transformers import pipeline

summarizer = pipeline("summarization", model=model, tokenizer=tokenizer, framework="tf")

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd

def calculate_tfidf(text, texts):
    tfidf = TfidfVectorizer()
    tfidf_matrix = tfidf.fit_transform(texts)
    feature_names = tfidf.get_feature_names_out()
    tfidf_values = tfidf.transform([text]).toarray()[0]
    tfidf_df = pd.DataFrame({'feature_names': feature_names, 'tfidf_values': tfidf_values})
    hashtags = tfidf_df.nlargest(5, 'tfidf_values').feature_names.to_numpy()
    result = [f"#{h}" for h in hashtags]
    return result


In [ ]:
def generate_summary_news(index=0,texto = None, outside = False):

  if outside:
    resumo = summarizer(
        texto,
        min_length=MIN_TARGET_LENGTH,
        max_length=1024,
      )[0]['summary_text']

    hashtags = calculate_tfidf(texto,raw_datasets['test']['text'])
    print(f"Texto:\n{texto}\n")
    print(f"Resumo:\n{resumo}\n")
    for item in hashtags:
      print(item, end=' ')

  else:
    resumo = summarizer(
          raw_datasets["test"][index]["text"],
          min_length=MIN_TARGET_LENGTH,
          max_length=256,
        )[0]['summary_text']
    
    hashtags = calculate_tfidf(raw_datasets['test'][index]['text'],raw_datasets['test']['text'])
    print(f"Texto:\n{raw_datasets['test'][index]['text']}\n")
    print(f"Resumo:\n{resumo}\n")
    for item in hashtags:
      print(item, end=' ')


In [ ]:
raw_datasets['test'][43]['text']

'Os ataques suicidas almejaram alvos britânicos em Istambul e deixaram 27 mortos e mais de 400 feridos. O cônsul-geral britânico, Roger Short, está entre os mortos, a maioria deles turcos. O Conselho de Segurança das Nações Unidas adotou, por unanimidade, uma resolução condenando os ataque em Istambul. O texto expressa condolências para "o povo e o governo da Turquia e da Grã-Bretanha", assim como para as famílias daqueles que foram mortos. Reações A resolução também expressa a determinação do Conselho de Segurança em combater todas as formas de terrorismo. Numa declaração feita anteriormente, o secretário-geral da ONU, Kofi Annan, disse que as pessoas que executaram os ataques mostraram sua falta de respeito pela vida humana. O primeiro-ministro italiano, Silvio Berlusconi, cujo país ocupa a presidência da União Européia, disse que "a estratégia de ódio e intimidação dos responsáveis pelos ataques não vai prevalecer". O secretário-geral da Otan (a aliança militar ocidental), George Ro

In [ ]:
text = "Até para a inteligência artificial (IA), segmento no qual as descobertas avançaram velozmente na última década, os últimos dois meses foram intensos. Em 30 de novembro, a OpenAI, uma startup americana fundada em 2015, tornou público o ChatGPT, uma ferramenta poderosa que inaugura uma nova fase na relação da humanidade com as máquinas e que traz implicações econômicas e sociais para os próximos anos. O ChatGPT é um robô de bate-papo (chatbot) gratuito capaz de produzir texto e trazer informações sobre assuntos diversos. A complexidade das respostas e a sofisticação da organização do texto chamam a atenção. Ele é capaz de produzir discursos de casamento, e-mails corporativos, textos em formato jornalístico, listas de organização e código de computação - é só saber pedir do jeito certo, inclusive em português. A capacidade deu notoriedade ao sistema, que passou a ser testado por milhões de pessoas. Segundo uma pesquisa da Similarweb, o ChatGPT alcançou 100 milhões de usuários ativos em janeiro, o que faz dele o serviço na internet com o crescimento de usuários mais veloz da história - atualmente é comum ele travar devido ao alto tráfego. Os casos de uso passaram a ser diversos. O professor Christian Terwiesch, da Universidade de Wharton, aplicou o exame final do Master in Business Administration (MBA) de sua universidade, além do Exame de Ordem (uma espécie de 'prova da OAB' nos EUA) e também do Exame de Licenciamento Médico dos Estados Unidos (USMLE). Em todos os casos, a máquina passou e reforçou os temores do mundo acadêmico sobre a necessidade de revisar os métodos de avaliação. Já o site de tecnologia Cnet colocou o ChatGPT para escrever textos - e foi criticado por não avisar os leitores nem remover erros factuais. O BuzzFeed foi além: anunciou que vai usar os algoritmos da OpenAI para produzir parte de seus conteúdos um mês depois que demitiu 180 pessoas. Enquanto isso, a revista Nature publicou um editorial no qual considera o ChatGPT uma ameaça para a ciência transparente. O ChatGPT não é o primeiro chatbot 'esperto' de que se tem notícia. No ano passado, o LaMDA, do Google, fez barulho quando um engenheiro da companhia afirmou que o sistema tinha desenvolvido consciência, algo refutado tanto pela empresa quanto por especialistas. O LaMDA, porém, nunca foi disponibilizado para uma grande audiência - startups como Character AI e Anthropic também têm ferramentas do tipo. Mas, graças à popularidade alcançada nas redes sociais, o ChatGPT surge como o maior exemplo de um novo capítulo na IA. 'O ChatGPT abre uma nova era. Até aqui, os sistemas otimizavam a rotulação de dados. Agora, a IA é capaz de gerar conteúdo inédito, o que amplia as possibilidades criativas da máquina', explica Anderson Soares, coordenador do Centro de Excelência em Inteligência Artificial da Universidade Federal de Goiás (UFG).Isso significa que os algoritmos que estão presentes na vida da maioria das pessoas, como o sistema do YouTube que sugere vídeos, era capaz de capturar tendências e fazer correlações a partir de dados já existentes. A nova geração de IA dá um passo além, criando novas informações - essa nova classe é conhecida como 'inteligência artificial gerativa'. Durante todo o ano passado, IAs gerativas ganharam espaço não apenas com a produção de texto. Em abril do ano passado, a OpenAI lançou o DALL-E 2, que gera imagens a partir de comandos de texto - o que inspirou uma geração de sistemas do tipo, como o Midjourney e o Stable Diffusion. Surgiram também sistemas capazes de gerar vídeo e até de clonar a voz humana. O ChatGPT serviu como o empurrão final para essa nova classe de sistemas. Não é só na produção de conteúdo que o ChatGPT representa avanço. A interface simples da ferramenta, no formato de um chatbot, possibilita a massificação de uma ferramenta poderosa. 'Esses novos sistemas possibilitam o acesso precoce à tecnologia, dispensando habilidades de programação', argumenta Soares. Até aqui, sistemas de IA e grandes bancos de dados trabalhavam sob a tutela de programadores, cientistas da computação, engenheiros de dados e designers. As IAs gerativas conseguem trabalhar para qualquer pessoa, independentemente dos seus conhecimentos técnicos em computação. O ChatGPT expõe a importância de grandes modelos de linguagem natural (LLM), bancos de dados gigantes usados para o processamento de texto escrito. Há alguns anos, o Google abriu a era dos LLM com o Bert, um sistema de compreensão de texto usado em produtos como a busca e o tradutor. Em 2020, a OpenAI chamou a atenção por tornar público o GPT-3 (abreviação de Generative Pre-trained Transformer), um modelo com 175 bilhões de parâmetros (representações matemáticas de padrões e tipos de texto) - o Bert tinha 110 milhões de parâmetros. Assim, o ChatGPT usa como 'cérebro' o GPT-3 (e a OpenAI já está trabalhando no sucessor, o GPT-4) e coloca nas mãos de usuários comuns um banco de dados poderoso, que só pode ser desenvolvido com mão de obra cara e infraestrutura computacional restrita. Em 2013, os pesquisadores Carl Benedikt Frey e Michael Osborne, da Universidade Oxford, publicaram um estudo no qual estimavam que 47% das profissões nos EUA seriam afetadas por tecnologias de automação, como IA e robótica. Agora, a profecia pode não apenas se realizar, como abranger um número ainda maior de profissionais. 'Todas as profissões que trabalham com texto serão afetadas pelo ChatGPT', afirma Edney Souza, professor de inovação, tecnologia e negócios digitais na Escola Superior de Propaganda e Marketing (ESPM). No mundo corporativo, cargos executivos e administrativos serão afetados. Áreas como educação, publicidade, jornalismo e direito também estão entre os alvos mais óbvios. Não é só isso. Como o ChatGPT também consegue escrever código e fazer contas, áreas como computação, engenharia, arquitetura e design também podem sentir os efeitos. Com o avanço das IAs gerativas de forma geral, qualquer profissão que também trabalha com imagem e vídeo poderá ser impactada - alguns especialistas falam que essa nova era pode ter impacto inédito em profissões do ensino superior. Um relatório de 2021 da Organização para a Cooperação e Desenvolvimento Econômico (OCDE) afirma que a adoção de IA pode aumentar as disparidades do mercado de trabalho entre aqueles que têm habilidade para usar IA e os que não têm. 'Garantir que os trabalhadores tenham as habilidades certas para trabalhar com novas tecnologia é, portanto, um desafio para a política (dos países)', diz o documento. Isso tornou o ChatGPT e as IAs gerativas em um dos principais assuntos discutidos no último Fórum Econômico Mundial, em Davos. Além do impacto econômico, a corrida da IA pode alterar as forças geopolíticas globais. '(IAs gerativas) serão extraordinariamente importantes para a competitividade global e para a segurança das nações', afirmou durante o evento Brad Smith, presidente da Microsoft e líder da companhia em IA. Os países estão buscando formas de regular e absorver a tecnologia - a Europa é a região mais avançada nas discussões de legislações específicas. Há outras questões no processo que são muito importantes, incluindo regulação, ética, viés, transparência, direito autoral e bem-estar social. O principal desafio é lidar com a velocidade sobre como as coisas se desenrolam. 'Esses sistemas ficarão ainda mais sofisticados, então é difícil dizer o que nos aguarda. Por enquanto, não é possível nem mesmo falar em estabilização das descobertas. Enquanto os LLM continuarem crescendo, teremos avanços ainda não imaginados', afirma Soares."

In [ ]:
generate_summary_news(index = 42)

Texto:
Os ataques suicidas almejaram alvos britânicos em Istambul e deixaram 27 mortos e mais de 400 feridos. O cônsul-geral britânico, Roger Short, está entre os mortos, a maioria deles turcos. O Conselho de Segurança das Nações Unidas adotou, por unanimidade, uma resolução condenando os ataque em Istambul. O texto expressa condolências para "o povo e o governo da Turquia e da Grã-Bretanha", assim como para as famílias daqueles que foram mortos. Reações A resolução também expressa a determinação do Conselho de Segurança em combater todas as formas de terrorismo. Numa declaração feita anteriormente, o secretário-geral da ONU, Kofi Annan, disse que as pessoas que executaram os ataques mostraram sua falta de respeito pela vida humana. O primeiro-ministro italiano, Silvio Berlusconi, cujo país ocupa a presidência da União Européia, disse que "a estratégia de ódio e intimidação dos responsáveis pelos ataques não vai prevalecer". O secretário-geral da Otan (a aliança militar ocidental), Geo

In [ ]:
generate_summary_news(texto = text, outside=True)

Texto:
Até para a inteligência artificial (IA), segmento no qual as descobertas avançaram velozmente na última década, os últimos dois meses foram intensos. Em 30 de novembro, a OpenAI, uma startup americana fundada em 2015, tornou público o ChatGPT, uma ferramenta poderosa que inaugura uma nova fase na relação da humanidade com as máquinas e que traz implicações econômicas e sociais para os próximos anos. O ChatGPT é um robô de bate-papo (chatbot) gratuito capaz de produzir texto e trazer informações sobre assuntos diversos. A complexidade das respostas e a sofisticação da organização do texto chamam a atenção. Ele é capaz de produzir discursos de casamento, e-mails corporativos, textos em formato jornalístico, listas de organização e código de computação - é só saber pedir do jeito certo, inclusive em português. A capacidade deu notoriedade ao sistema, que passou a ser testado por milhões de pessoas. Segundo uma pesquisa da Similarweb, o ChatGPT alcançou 100 milhões de usuários ativo